## На рандоме взял страницу с Chanel 

In [2]:
import pandas as pd
import requests
from lxml import html
import xml.etree.ElementTree as ET
from time import sleep
from tqdm import tqdm
import json

In [3]:
#page = requests.get('https://www.fragrantica.ru/designers/Chanel.html')
#data = html.fromstring(page.content)

#### Сайт попался <b>вредный</b>, выдаёт код 429. Я как <s>умный</s> русский человек загуглил и увидел, что это значит слишком много запросов. Поэтому будем обманывать.

In [3]:
url = 'https://www.fragrantica.ru/designers/Chanel.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/39.0.2171.95 Safari/537.36'}

##### Я знаю, про фэйк юзер эджент, но он почему-то у меня не работает, поэтому пришлось так

In [4]:
page = requests.get(url, headers=headers)
data = html.fromstring(page.content)

In [5]:
page.status_code

200

In [6]:
aromas_list = data.xpath('//div[@class="perfumeslist"]')
gender = data.xpath('//span[@class="mtext"]')

##### Видим, что тут всё довольно однообразно. Берём 1 духи и проверям 

In [11]:
for item in aromas_list[0].text_content().split('\n'):
    if item != '':
        print(item.strip())

Antaeus
1981 для мужчин


In [12]:
for catalog in data.xpath('//div[@class="perfumeslist"]')[0:]:
    print(catalog.text_content().replace('\n',''))

 Antaeus1981 для мужчин
 Bois Noir1987 для мужчин
 Chanel No 461946 для женщин
 Cuir de Russie Parfum1924 для женщин и мужчин
 Gabrielle2017 для женщин
 Le 1940 Beige de Chanel1931 для женщин
 Le 1940 Bleu de Chanel1931 для женщин
 Le 1940 Rouge de Chanel1931 для женщин
 Une Fleur de Chanel1998 для женщин
 Allure1996 для женщин
 Allure eau de parfum1999 для женщин
 Allure Eau Fraichissante Pour l`Ete2002 для женщин
 Allure Hair Mist для женщин
 Allure Homme Eau Fraichissante Pour l`Ete2002 для мужчин
 Allure Homme Edition Blanche2008 для мужчин
 Allure Homme Edition Blanche Eau de Parfum2014 для мужчин
 Allure Homme Sport2004 для мужчин
 Allure Homme Sport Cologne2007 для мужчин
 Allure Homme Sport Eau Extreme2012 для мужчин
 Allure Parfum для женщин
 Allure Pour Homme1999 для мужчин
 Allure Sensuelle2005 для женщин
 Allure Sensuelle Eau de Toilette2006 для женщин
 Allure Sensuelle Parfum2006 для женщин
 Bleu de Chanel2010 для мужчин
 Bleu de Chanel Eau de Parfum2014 для мужчин
 Chance

#####  Получили список духов с этой страницы, с разделением на М/Ж/Унисекс, но пока только в строке 

In [15]:
def getData(data):
    voc = {}
    for count,catalog in enumerate(data.xpath('//div[@class="perfumeslist"]')[0:]):
        for line in catalog.xpath('div/div/p'):
            A = {}
            for a in line.xpath('a'):
                A['link'] = a.attrib.get('href')
                A['title'] = a.text_content().replace('\n',',')
            for span in line.xpath('span'):
                mtext = span.text_content().replace('для',';').split(';')
                A['year'] = mtext[0].strip()
                A['sex'] = mtext[1].strip()
            voc[count] = A
    return pd.DataFrame.from_dict(voc, orient ='index')

#####  Пишем фунцию для забора инфы о духах - название, год и пол. Год и пол лежат в одном, пока хз как разделить 

In [16]:
perfumes = getData(data)

#####  И тут заметил, что <b>всё не так просто.</b> Имею в виду, что название духов не выведено в отдельный блок по всем духам. Названию каждый духой соответствует линк на них. Пока хз как это поправить, чтобы вместо отдельного линка у каждых, был бы единый столбец 'name', подобно 'mtext'.

In [17]:
df = perfumes.copy()
df.dropna()
df.head()

,link,title,year,sex
0,/perfume/Chanel/Antaeus-616.html,Antaeus,1981,мужчин
1,/perfume/Chanel/Bois-Noir-34568.html,Bois Noir,1987,мужчин
2,/perfume/Chanel/Chanel-No-46-22520.html,Chanel No 46,1946,женщин
3,/perfume/Chanel/Cuir-de-Russie-Parfum-47794.html,Cuir de Russie Parfum,1924,женщин и мужчин
4,/perfume/Chanel/Gabrielle-43718.html,Gabrielle,2017,женщин


##### Вполне логично, что пандас <s> умнее меня </s> делает для каждого уникального линка уникальный столбец, тем самым получаем 105 колонок)))))) 

In [18]:
df.shape

(105, 4)